<a href="https://colab.research.google.com/github/nguyenduongtri5703/sentiment_analysis/blob/main/Sentiment_Analysis_Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd '/content/gdrive/MyDrive/model_history'

Mounted at /content/gdrive
/content/gdrive/MyDrive/model_history


In [ ]:
!wget -q -nc https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip
!./ngrok authtoken 2Toy2x79wVRXDSoX0w4a4aDNL7N_4Jz2Z3As1hveLnwNp25Kb

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from flask import Flask, request, jsonify
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F
import threading

app = Flask(__name__)

model_path = "/content/gdrive/MyDrive/model_history"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

label_map = {
    0: "tiêu cực",
    1: "trung lập",
    2: "tích cực"
}

@app.route('/analyze', methods=['POST'])
def analyze():
    data = request.get_json()
    text = data.get("text", "")

    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
        probs = F.softmax(outputs.logits, dim=1)
        predicted = torch.argmax(probs, dim=1).item()
        confidence = probs[0][predicted].item()

    return jsonify({
        "sentiment": label_map[predicted],
        "confidence": round(confidence, 4)
    })

def run_app():
    app.run(host="0.0.0.0", port=5000)

thread = threading.Thread(target=run_app)
thread.start()


 * Serving Flask app '__main__'
 * Debug mode: off


In [ ]:
import time
import requests

# Chạy ngrok dưới nền
get_ipython().system_raw('./ngrok http 5000 &')

# Đợi ngrok khởi động
time.sleep(3)

# Lấy public URL
r = requests.get("http://localhost:4040/api/tunnels")
public_url = r.json()['tunnels'][0]['public_url']
print("📡 Public URL:", public_url)

📡 Public URL: https://3151-34-83-44-99.ngrok-free.app


In [ ]:
import json

# Gắn url in ra ở bên trên
url = "https://3151-34-83-44-99.ngrok-free.app/analyze"
data = {
    "text": "Sản phẩm tốt"
}
response = requests.post(url, json=data)
print(response.json())

INFO:werkzeug:127.0.0.1 - - [15/Jun/2025 08:37:35] "POST /analyze HTTP/1.1" 200 -


{'confidence': 0.9931, 'sentiment': 'tích cực'}
